In [3]:
import logging
import pandas as pd
import ccxt
from datetime import datetime

pd.set_option('display.float_format', '{:.5f}'.format)


def fetch_prices(binance, symbol_one, symbol_two, look_back_days=1080):
    # Prepare empty dataframe for results
    df_final = pd.DataFrame()
    
    # Calculate the end timestamp for the data fetching
    end_timestamp = binance.milliseconds()
    since_timestamp = end_timestamp - look_back_days * 24 * 60 * 60 * 1000  # look back days in milliseconds
    
    while since_timestamp < end_timestamp:
        try:
            # Fetch OHLCV data
            ohlcv_theta = binance.fetch_ohlcv(symbol_two, '1h', since=since_timestamp, limit=1000)
            ohlcv_xrp = binance.fetch_ohlcv(symbol_one, '1h', since=since_timestamp, limit=1000)
            
            # Create temporary dataframes to hold fetched data
            df_theta_temp = pd.DataFrame(ohlcv_theta, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df_xrp_temp = pd.DataFrame(ohlcv_xrp, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            
            # Merge dataframes
            df_temp = pd.merge(df_theta_temp[['timestamp', 'close']], df_xrp_temp[['timestamp', 'close']], on='timestamp', how='inner')
            df_temp.columns = ['timestamp', symbol_two, symbol_one]
            
            # Concatenate with the final dataframe
            df_final = pd.concat([df_final, df_temp])
            
            # Update the since_timestamp to the last timestamp fetched
            since_timestamp = df_final['timestamp'].max() + (60 * 60 * 1000)  # Add one hour

        except Exception as e:
            logging.error(f"Error in fetching prices: {e}")
            break
    
    # Remove duplicates and sort the dataframe
    df_final.drop_duplicates(subset='timestamp', keep='first', inplace=True)
    df_final.sort_values(by='timestamp', inplace=True)
    
    return df_final

# Set up exchange connection
binance = ccxt.binance()

# Fetch prices
prices = fetch_prices(binance, "XRPUSDT", "THETAUSDT")
print(prices.head())


# Convert timestamps to datetime format and set as index
prices['time'] = pd.to_datetime(prices['timestamp'], unit='ms')
prices.set_index('time', inplace=True)

# Save to CSV
prices.to_csv('XRPTHETA_yearly.csv', index=True, float_format='%.5f')


       timestamp  THETAUSDT  XRPUSDT
0  1606183200000    0.68746  0.59974
1  1606186800000    0.70396  0.63776
2  1606190400000    0.72465  0.68505
3  1606194000000    0.71359  0.73306
4  1606197600000    0.70433  0.69859
